In [1]:
import torch
from time import time
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from zipfile import ZipFile
import numpy as np
import pandas as pd
from time import time
import zipfile
import re
import os
import torchvision
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.io import read_image
from matplotlib import image

In [2]:
from vgg_pretrained import vgg_all_freezed,total_trainable_parameters,full_vgg,vgg_layer_freeze
from prepare_dataset import MyDataset,subset_generator
from vgg_pretrained import vgg_all_freezed,total_trainable_parameters,full_vgg,vgg_layer_freeze
from train_fn import train,validation,image_validation,validation_classes,train_batch
from Model import Network,new_model,new_Regressor

In [3]:
fullvgg=full_vgg()

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose([transforms.ToTensor(),normalize])
target_transform=transforms.Compose([transforms.ToTensor()])

anno_dir=r"C:\Users\Engineer Protobetz\Documents\Programs Directory\data\AffectNet\train_set\annotations"
image_dir=r"C:\Users\Engineer Protobetz\Documents\Programs Directory\data\AffectNet\train_set\images"
train_data=MyDataset(anno_dir,image_dir,transform=transform, target_transform=None)

val_anno_dir=r"C:\Users\Engineer Protobetz\Documents\Programs Directory\data\AffectNet\val_set\annotations"
val_image_dir=r"C:\Users\Engineer Protobetz\Documents\Programs Directory\data\AffectNet\val_set\images"
val_data=MyDataset(val_anno_dir,val_image_dir,transform=transform, target_transform=None)

In [4]:
batch = 512
train_loader=torch.utils.data.DataLoader(train_data,
                                         batch_size=batch,
                                         shuffle=True,
                                         num_workers=4,
                                         pin_memory=True,
                                         sampler=None)
val_loader=torch.utils.data.DataLoader(val_data,
                                       batch_size=batch,
                                       shuffle=False,
                                       num_workers=4,
                                       pin_memory=True,
                                      sampler=None)
val_class_loader=torch.utils.data.DataLoader(val_data,
                                       batch_size=1,
                                       shuffle=False,
                                       num_workers=4,
                                       pin_memory=True,
                                      sampler=None)

In [5]:
print(train_data.__getitem__(9))

(tensor([[[ 1.1015,  1.1872,  1.0673,  ...,  0.4166,  0.4679,  0.5364],
         [ 1.1700,  1.2043,  0.9988,  ...,  0.4679,  0.4679,  0.5022],
         [ 1.2043,  1.2214,  0.9474,  ...,  0.4337,  0.3994,  0.3994],
         ...,
         [-0.1486,  0.0569,  0.2796,  ..., -1.1760, -1.1589, -1.1760],
         [-0.2342, -0.0458,  0.1768,  ..., -1.1760, -1.1932, -1.2274],
         [-0.2684, -0.0972,  0.1083,  ..., -1.1760, -1.2274, -1.2788]],

        [[ 1.2206,  1.2731,  1.1681,  ...,  0.4503,  0.4853,  0.5728],
         [ 1.2556,  1.2906,  1.0980,  ...,  0.5028,  0.4853,  0.5378],
         [ 1.2906,  1.3256,  1.0455,  ...,  0.4678,  0.4153,  0.4328],
         ...,
         [-0.6001, -0.3901, -0.1625,  ..., -1.0028, -0.9853, -1.0028],
         [-0.6877, -0.4951, -0.2675,  ..., -1.0028, -1.0203, -1.0553],
         [-0.7227, -0.5476, -0.3375,  ..., -1.0028, -1.0553, -1.1078]],

        [[ 1.2108,  1.2805,  1.1237,  ...,  0.2173,  0.1999,  0.2348],
         [ 1.2631,  1.2980,  1.0539,  ...,  

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [7]:
feature_extractor=vgg_layer_freeze(17)
total_trainable_parameters(feature_extractor)


Total trainable parameters of Sequential is 	 12979200


In [8]:

#input_features_for_denselayer=get_output_features_of_model(model=feature_extractor,batch_size=32,dataloader=train_loader)
input_features_for_denselayer=25088


In [9]:
modelReg=new_Regressor(feature_extractor,input_features_for_denselayer).to(device)
total_trainable_parameters(modelReg)


Total trainable parameters of new_Regressor is 	 222705155


In [10]:
loss_plain=nn.L1Loss()
optimizer=optim.NAdam(modelReg.parameters(),lr=0.0001)
classes = ['Valance', 'Arousal']
classes_exp=['Neutral','Happy','Sad','Surprise','Fear','Disgust','Anger','Contempt']

In [11]:
epochs = 1
for epoch in range(epochs):
    print(f'Epoch-{epoch}')
    train_batch(train_loader,modelReg,loss_plain,optimizer)
    validation(val_loader,modelReg,loss_plain)

Epoch-0


  0%|                                                                                          | 0/562 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([512, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([512, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
  0%|                                                                                          | 0/562 [03:09<?, ?it/s]


RuntimeError: The size of tensor a (2) must match the size of tensor b (512) at non-singleton dimension 1